In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import gensim as gs
import math
import numpy as np
import tqdm

dct_title = gs.corpora.Dictionary.load ("../result/dct_body.dump")

corpus = gs.corpora.MmCorpus('../result/corpus_body_only.mm')

PARAM_K1 = 1.5
PARAM_B = 0.75
EPSILON = 0.25

corp_size = 0
corp_size_words = 0
for doc in tqdm.tqdm (corpus):
    corp_size_words += sum (dict (doc).itervalues ())
    corp_size += 1

avgdl = float (corp_size_words) / corp_size

idfs = {}

summ_idf = 0


for word_id, freq in dct_title.dfs.iteritems ():
    idfs[word_id] = math.log(corp_size - freq + 0.5) - math.log(freq + 0.5)
    summ_idf += idfs[word_id]

average_idf = float (summ_idf) / len (dct_title.dfs)

for id in idfs.iterkeys ():
    idfs[id] = idfs[id] if idfs[id] >= 0 else EPSILON * average_idf

q_ids = []
queries = []
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").upper ().split ("\t")
        q_ids.append (int (q[0]))
        queries.append (dict (dct_title.doc2bow (q[1].split ())))

q_size = len (queries)

result = np.zeros ((q_size, corp_size))

2018-07-04 11:53:06,197 : INFO : 'pattern' package not found; tag filters are not available for English
2018-07-04 11:53:06,321 : INFO : loading Dictionary object from ../result/dct_body.dump
2018-07-04 11:53:30,602 : INFO : loaded ../result/dct_body.dump
2018-07-04 11:53:30,765 : INFO : loaded corpus index from ../result/corpus_body_only.mm.index
2018-07-04 11:53:30,766 : INFO : initializing cython corpus reader from ../result/corpus_body_only.mm
2018-07-04 11:53:30,796 : INFO : accepted corpus with 582167 documents, 29234379 features, 1683925518 non-zero entries
100%|██████████| 582167/582167 [16:26<00:00, 590.08it/s]


In [26]:
result = np.zeros ((q_size, corp_size))

In [27]:
for doc_i, doc in tqdm.tqdm (enumerate (corpus)):
    doc = dict (doc)
    doc_keys = set (doc)
    doc_len = sum (doc.itervalues ())
    for q_i, q in enumerate (queries):
        score = 0
        for word in set (q.keys ()) & doc_keys:
            idf = idfs[word]
            score += (idf * doc[word] * (PARAM_K1 + 1)
                      / (doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len / avgdl)))
        result[q_i, doc_i] = score

50it [00:02, 19.18it/s]Exception KeyError: KeyError(<weakref at 0x7f57eda77a48; to 'tqdm' at 0x7f58281d1ed0>,) in <bound method tqdm.__del__ of 29it [00:10,  2.73it/s]> ignored
582167it [3:58:47, 40.63it/s]


In [28]:
with open ("../result/result_bm25_body_self_0.csv", "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for q_num, qid in tqdm.tqdm (enumerate (q_ids)):
        for doc in np.argsort (result[q_num, :])[-5:][::-1]:
            fout.write ("{},{}\n".format (qid, doc))

6311it [04:48, 21.85it/s]
